# Running the model

Setup a `feisty` integration.


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import yaml

import feisty

## Configure testcase to run

TODO: this is using the first column (which I believe is shallowest); want to add deepest column as well.
That will require refactoring the yaml file and also changing test_ds.

In [3]:
def _comparison(matlab_vals, py_vals, row_name, full_table, seps, thres):
    nstep = np.min([len(matlab_vals["time"].data), len(py_vals["time"].data)])
    nx = np.min([len(matlab_vals["X"].data), len(py_vals["X"].data)])
    matlab_vals = matlab_vals.isel(time=slice(0, nstep), X=slice(0, nx)).data
    py_vals = py_vals.isel(time=slice(0, nstep), X=slice(0, nx)).data
    if thres:
        matlab_vals = np.where(np.abs(matlab_vals) > thres, matlab_vals, 0)
        py_vals = np.where(np.abs(py_vals) > thres, py_vals, 0)
    rel_err_denom = np.where(
        matlab_vals != 0, matlab_vals, 1
    )  # avoid dividing by 0 in next np.where() statement
    if np.any(np.isnan(py_vals)):
        print(f"python has nans for {rowname}!")
        return
    rel_errs = np.where(matlab_vals != 0, np.abs((py_vals - matlab_vals) / rel_err_denom), np.nan)
    rel_errs = np.where(np.logical_and(np.isnan(rel_errs), py_vals == 0), 0, rel_errs)
    rel_errs = np.where(np.logical_and(np.isnan(rel_errs), py_vals > 0), np.inf, rel_errs)
    rel_errs = np.where(np.logical_and(np.isnan(rel_errs), py_vals < 0), -np.inf, rel_errs)

    if full_table or rel_err > 1e-12:
        max_inds = np.nonzero(np.abs(rel_errs) == np.max(np.abs(rel_errs)))
        t_ind = max_inds[0][0]
        x_ind = max_inds[1][0]
        matlab_val = matlab_vals[t_ind][x_ind]
        py_val = py_vals[t_ind][x_ind]
        rel_err = rel_errs[t_ind][x_ind]
        print(
            f"{seps[0]}{row_name} (t={t_ind}, X={x_ind}){seps[1]}{matlab_val:10.4e}{seps[2]}"
            + f"{py_val:10.4e}{seps[2]}{rel_err:10.4e}{seps[-1]}"
        )


def compare_nc(baseline_ds, test_da_or_ds, full_table=True, markdown_formatting="true", thres=None):
    if type(test_da_or_ds) == xr.Dataset:
        ds = test_da_or_ds
        da = None
        baseline_da = None
        table_header = ''
    else:
        da = test_da_or_ds
        ds = None
        da_dims = ["group", "fish"]
        da_dim = ''
        for dad in da_dims:
            if dad in da.dims:
                da_dim = dad
                break
        baseline_da = baseline_ds[da.name]
        table_header = da_dim
    if markdown_formatting:
        print(f"| {table_header} | Matlab Value | Python Value | Rel Err |")
        print("| --- | --- | --- | --- |")
        seps = ["| ", " | ", " | ", " |"]
    else:
        seps = ["", ": ", ", ", ""]
    if da is not None:
        for n, dimname in enumerate(da[da_dim]):
            _comparison(
                baseline_da.isel({da_dim: n}),
                da.isel({da_dim: n}),
                dimname.data,
                full_table,
                seps,
                thres,
            )
    else:
        for varname in ds:
            _comparison(
                baseline_ds.isel(zooplankton=0)[varname],
                ds.isel(zooplankton=0)[varname],
                varname,
                full_table,
                seps,
                thres,
            )

In [4]:
# matlab_script = "test_case" ; force_nonnegative=True
matlab_script, force_nonnegative = "test_locs3", True
matlab_script, force_nonnegative = "FOSI_cesm", False

baselines_from_nc = xr.open_dataset(f'../matlab_baselines/{matlab_script}.nc')

nyears = 1
settings_in = {}
if matlab_script == "test_case":
    testcase = feisty.config_testcase("tanh_shelf", "cyclic")
elif matlab_script in ["test_locs3", "FOSI_cesm"]:
    is_locs3 = matlab_script == "test_locs3"
    if is_locs3:
        nyears = 2
    else:
        # only changing benthic_prey.defaults.carrying_capacity, food_web[0]['preference'], food_web[1]['preference'], and food_web[2]['preference']
        # To say this UI needs work is an understatement
        settings_in['benthic_prey'] = {
            'defaults': {'benthic_efficiency': 0.075, 'carrying_capacity': 80},
            'members': [{'name': 'benthic_prey'}],
        }
        settings_in['food_web'] = [
            {'predator': 'Sf', 'prey': 'Zoo', 'preference': 0.9},
            {'predator': 'Sp', 'prey': 'Zoo', 'preference': 0.9},
            {'predator': 'Sd', 'prey': 'Zoo', 'preference': 0.9},
            {'predator': 'Mf', 'prey': 'Zoo', 'preference': 0.45},
            {'predator': 'Mf', 'prey': 'Sf', 'preference': 1.0},
            {'predator': 'Mf', 'prey': 'Sp', 'preference': 1.0},
            {'predator': 'Mf', 'prey': 'Sd', 'preference': 1.0},
            {'predator': 'Mp', 'prey': 'Zoo', 'preference': 0.45},
            {'predator': 'Mp', 'prey': 'Sf', 'preference': 1.0},
            {'predator': 'Mp', 'prey': 'Sp', 'preference': 1.0},
            {'predator': 'Mp', 'prey': 'Sd', 'preference': 1.0},
            {'predator': 'Md', 'prey': 'benthic_prey', 'preference': 1.0},
            {'predator': 'Lp', 'prey': 'Mf', 'preference': 0.5},
            {'predator': 'Lp', 'prey': 'Mp', 'preference': 1.0},
            {'predator': 'Lp', 'prey': 'Md', 'preference': 1.0},
            {'predator': 'Ld', 'prey': 'Mf', 'preference': 0.375},
            {'predator': 'Ld', 'prey': 'Mp', 'preference': 0.75},
            {'predator': 'Ld', 'prey': 'Md', 'preference': 1.0},
            {'predator': 'Ld', 'prey': 'benthic_prey', 'preference': 1.0},
        ]
    testcase = feisty.config_from_netcdf(
        './forcing.yaml',
        matlab_script,
        ignore_year_in_forcing=is_locs3,  # True,
        settings_in=settings_in,
    )
else:
    raise ValueError(f"unknown matlab_script '{matlab_script}'")

# Set negative forcing values to zero
if force_nonnegative:
    for var in ['poc_flux_bottom', 'zooC', 'zoo_mort']:
        baselines_from_nc[var].data = np.where(
            baselines_from_nc[var].data > 0, baselines_from_nc[var].data, 0
        )

In [5]:
if testcase.ignore_year:
    new_forcing = testcase.forcing.isel(time=slice(1, -1)).transpose('zooplankton', 'time', 'X')
else:
    new_forcing = testcase.forcing.transpose('zooplankton', 'time', 'X')
compare_nc(
    baselines_from_nc,
    new_forcing,
)

print(
    f"\nMax diff in depth: {np.max(np.abs(baselines_from_nc['dep'].data - testcase.obj.domain_dict['bathymetry'].data))}"
)

|  | Matlab Value | Python Value | Rel Err |
| --- | --- | --- | --- |
| T_pelagic (t=0, X=0) | -1.6900e+00 | -1.6900e+00 | 0.0000e+00 |
| T_bottom (t=0, X=0) | -1.7382e+00 | -1.7382e+00 | 0.0000e+00 |
| poc_flux_bottom (t=0, X=0) | 3.0023e-01 | 3.0023e-01 | 0.0000e+00 |
| zooC (t=0, X=0) | 9.6238e+00 | 9.6238e+00 | 0.0000e+00 |
| zoo_mort (t=0, X=0) | 8.1119e-01 | 8.1119e-01 | 0.0000e+00 |

Max diff in depth: 0.0


## Run the model


In [6]:
nsteps = nyears * 365
make_plot = False
%time testcase.run(nsteps)

CPU times: user 5min, sys: 18min 26s, total: 23min 26s
Wall time: 1h 33min 51s


In [7]:
if nsteps > 1 and make_plot:
    for group in testcase.ds.group.data:
        testcase.ds.biomass.sel(group=group).isel(X=0).plot()
    if matlab_script == "test_case":
        plt.ylim([1e-6, 1])
    if matlab_script == "test_locs3":
        plt.ylim([5e-6, 50])
    plt.yscale("log")
    plt.legend(testcase.ds.group.data)

In [8]:
compare_nc(baselines_from_nc, testcase.ds["biomass"], nsteps, thres=1e-300)

| group | Matlab Value | Python Value | Rel Err |
| --- | --- | --- | --- |
| Sf (t=258, X=60616) | 1.5367e-02 | 1.5367e-02 | 2.9886e-11 |
| Sp (t=138, X=799) | 3.0873e-26 | 3.0873e-26 | 1.4703e-10 |
| Sd (t=258, X=60616) | 3.2256e-05 | 3.2256e-05 | 2.9868e-11 |
| Mf (t=364, X=72257) | 2.5577e+00 | 2.5577e+00 | 5.8565e-13 |
| Mp (t=248, X=12340) | 3.0945e-01 | 3.0945e-01 | 6.2886e-12 |
| Md (t=330, X=2822) | 6.4599e-01 | 6.4599e-01 | 2.4319e-13 |
| Lp (t=284, X=54698) | 9.8967e-205 | 9.8967e-205 | 4.0915e-11 |
| Ld (t=257, X=39946) | 4.9865e+00 | 4.9865e+00 | 5.5929e-14 |
| benthic_prey (t=134, X=77347) | 5.0557e-01 | 5.0557e-01 | 2.1016e-13 |
